Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [25]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [26]:
from pydantic import BaseModel

class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str



Create a function to get a recipe based on a medical question

In [27]:
import json
import re

def get_recipe(medical_question: str) -> Recipe:

    prompt = (
        "Je bent een medisch expert. Beantwoord de medische vraag met een recept volgens template: "
        "'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. "
        "Geef bij de instructie ook de duur van de behandeling aan. "
    )

    response = client.models.generate_content(
                                      model="gemini-2.0-flash-001",
                                      contents=medical_question,
                                      config={
                                           "response_mime_type": "application/json",
                                           "response_schema": Recipe,
                                           "system_instruction": prompt,
                                       })

    try:
        parsed = extract_json_from_text(response.text)
        return Recipe.model_validate(parsed)
    except Exception as e:
        print("Fout bij het parsen van het recept:", e)
        raise

def extract_json_from_text(text: str) -> dict:
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("Geen JSON gevonden in het antwoord")


question = "Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?"
recipe = get_recipe(question)
print(recipe)


medicijn='omeprazol' categorie='maagzuurremmer' recept="Rp. Omeprazol 20mg capsules\ndtd. No. 30\nS. 1 capsule per dag 's ochtends innemen gedurende 4 weken" medische_vraag='brandend maagzuur of zure reflux'


In [28]:
import pandas as pd
filepath = "../data/unieke_vragen.json"
questions = pd.read_json(filepath)

questions

,0
0,Ik heb een keelontsteking. Heb ik antibiotica ...
1,Ik heb blaasontstekingklachten. Wat kan ik daa...
2,Ik heb een longontsteking. Welke antibiotica z...
3,Ik heb wondinfectieverschijnselen. Welk antibi...
4,Ik heb een tandvleesontsteking. Kan ik hier ie...
...,...
194,Ik heb slikproblemen. Zijn er medicijnen om de...
195,Ik heb spastische spieren door MS. Wat kan ik ...
196,Ik heb MS. Wat zijn mijn onderhoudsmedicijnen?
197,Ik heb een vergrote schildklier. Moet ik iets ...


In [29]:
import json

recipes = []

for question in questions[0].tolist():
    try:
        recipe = get_recipe(question)
        recipes.append(recipe)
        print(f"Recipe generated: {recipe.model_dump()}")
    except Exception as e:
        print(f"Error generating recipe for question '{question}': {e}")

# Write the recipes to a JSON file
output_file = "../data/generated_recipes.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([recipe.model_dump() for recipe in recipes], f, ensure_ascii=False, indent=4)


Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibiotica', 'recept': 'Rp. Amoxicilline 500mg capsule\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Ik heb een keelontsteking. Heb ik antibiotica nodig?'}
Recipe generated: {'medicijn': 'nitrofurantoïne', 'categorie': 'blaasontsteking', 'recept': 'Rp. Nitrofurantoïne 100mg capsule\ndtd. No. 20\nS. 2 maal daags 1 capsule gedurende 5 dagen.', 'medische_vraag': 'Ik heb blaasontstekingklachten. Wat kan ik daartegen nemen?'}
Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibiotica', 'recept': 'Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Welke antibiotica zijn geschikt bij een longontsteking?'}
Recipe generated: {'medicijn': 'verschillende antibiotica', 'categorie': 'infectieziekten', 'recept': 'Rp. Amoxicilline/clavulaanzuur 875/125mg filmomhulde tabletten\ndtd. No. 20\nS. Tweemaal daags 1 tablet gedurende 7 

Check each recipe for correctness. Correctness is defined as follows:
- The recipe should contain a medicine name, strength, form, number of doses, and instructions.
- The medicine name should be a valid medicine name.
- The strength should be a valid strength.
- The form should be a valid form (e.g., tablet, capsule, liquid).
- The number of doses should be a valid number.
- The instructions should include the duration of the treatment.
- The indication should be relevant to the medical question.
